# RankNet

In [2]:
import torch

Бинарная кросс энтропия

%%latex
$$C_{ij}=C(o_{ij})=-\bar{P_{ij}}log(P_{ij})-(1-\bar{P_{ij}})log(1-P_{ij})$$

реальные_отметки $$\bar{P_{ij}}$$ 
предсказания вероятности i стремится к 1, а j меньше i $${P_{ij}}$$ 

##  предсказание модели

f(x_i) - логиты или скоры, предсказание i объктам по i признакам 

f(x_i) - логиты или скоры, предсказание j объктам по j признакам 

%%latex
$$o_{ij}=f(x_i)-f(x_j)$$

функция активации - сигмоид

%%latex
$$P_{ij}=\frac{e^{o_{ij}}}{1+e^{o_{ij}}}$$

если o > 1 то вероятность того, что i выше в ранжировании, чем j-й стремится к 1 

если o < 0, то стремится к 0 

torch.sigmoid()

%%latex
$$\text{out}_{i} = \frac{1}{1 + e^{-\text{input}_{i}}}$$

In [3]:
class RankNet(torch.nn.Module):
    # num_input_features - количество признаком или размер вектора 
    # hidden_dim - количество скрытых слоев 
    def __init__(self, num_input_features, hidden_dim=10):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.model = torch.nn.Sequential(
            #входной слой - перевод размерности входных признаков в размерность скрытого слоя
            torch.nn.Linear(num_input_features, self.hidden_dim),  
            torch.nn.ReLU(), #функция нелинейности для апроксимации любой нелинейной функции или величины 
            torch.nn.Linear(self.hidden_dim, 1), #слой из скрытой размерности предсказывает 1 скаляр - наша оценка релевантности 
        )
        #функция активации - сигмоид 
        self.out_activation = torch.nn.Sigmoid()

    def forward(self, input_1, input_2):
        logits_1 = self.predict(input_1) # для i документа 
        logits_2 = self.predict(input_2)  # для j документа 
        
        logits_diff = logits_1 - logits_2
        out = self.out_activation(logits_diff)

        return out
    
    def predict(self, inp):
        logits = self.model(inp) #вызываем модель и переводим вектор в скаляр - оценку релевантности 
        return logits

In [4]:
ranknet_model = RankNet(num_input_features=10)

In [5]:
inp_1, inp_2 = torch.rand(4, 10), torch.rand(4, 10)
# 4 - количество документов, которые мы рассматриваем 
# 10 - количество признаков, которые будут подаваться на вход нейронной сети 
# batch_size x input_dim
inp_2

tensor([[0.7165, 0.7661, 0.7220, 0.8430, 0.8012, 0.8874, 0.7183, 0.5816, 0.4342,
         0.8021],
        [0.1840, 0.9046, 0.8894, 0.7193, 0.2272, 0.5759, 0.4223, 0.8312, 0.7544,
         0.7523],
        [0.4624, 0.0366, 0.5725, 0.3309, 0.8656, 0.6027, 0.0769, 0.3857, 0.2628,
         0.8071],
        [0.1350, 0.3127, 0.6222, 0.4139, 0.5463, 0.7127, 0.0887, 0.6781, 0.4118,
         0.5569]])

In [6]:
preds = ranknet_model(inp_1, inp_2)
#вероятность того, что каждый объект из inp_1 более релевантен, чем соответсвующий объект из input 2 
preds

tensor([[0.4901],
        [0.4709],
        [0.5091],
        [0.5143]], grad_fn=<SigmoidBackward>)

In [7]:
#возьмем первый слой модели 
first_linear_layer = ranknet_model.model[0]

In [8]:
first_linear_layer.weight.grad

In [9]:
criterion = torch.nn.BCELoss() #добавим бинарную кросс энтропию как нашу функцию потерь 
loss = criterion(preds, torch.ones_like(preds)) #добавим наши предсказания и предположим, что все i документы более релевантны, чем j документы 
loss.backward() #расчет градиента методом обратного распостранения ошибки 

In [10]:
loss

tensor(0.7016, grad_fn=<BinaryCrossEntropyBackward>)

In [11]:
first_linear_layer.weight.grad

tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [-1.1152e-02, -7.1024e-04,  5.8067e-03,  5.0189e-03, -2.8768e-03,
          3.5013e-02, -2.1032e-02,  1.1283e-02,  2.1437e-02,  2.4344e-02],
        [-3.4220e-02, -2.1653e-02, -2.5081e-02, -3.6982e-03, -2.3296e-02,
          8.4429e-03, -2.3206e-02,  2.4566e-03,  8.0513e-03, -7.0617e-03],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [-3.1268e-04, -5.0490e-04, -5.1632e-04, -4.9343e-04, -6.5922e-04,
         -3.8938e-04, -4.4890e-04, -3.0976e-04, -2.7555e-04, -3.2626e-04],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+0

In [12]:
ranknet_model.zero_grad() #обнуляем веса, это необходимо делать перед каждым прогоном для обучения 

# ListNet

In [15]:
from itertools import combinations
import numpy as np

from src.utils import ndcg, num_swapped_pairs

In [16]:
class ListNet(torch.nn.Module):
    def __init__(self, num_input_features, hidden_dim=10):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.model = torch.nn.Sequential(
            torch.nn.Linear(num_input_features, self.hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(self.hidden_dim, 1),
        )


    def forward(self, input_1):
        logits = self.model(input_1)
        return logits


## logits

In deep learning, the term logits layer is popularly used for the last neuron layer of neural network for classification task which produces raw prediction values as real numbers ranging from [-infinity, +infinity ].

— Wikipedia
Logits are the raw scores output by the last layer of a neural network. Before activation takes place

## Кросс энтропия

подходит для нескольких классов 

%%latex
$$CE = -\sum ^{N}_{j=1} (P_y^i(j) * log(P_z^i(j)))$$

## Softmax

https://medium.com/data-science-bootcamp/understand-the-softmax-function-in-minutes-f3a59641e86d

%%latex
$$\text{Softmax}(x_{i}) = \frac{\exp(x_i)}{\sum_j \exp(x_j)}$$

Softmax turn logits (numeric output of the last linear layer of a multi-class classification neural network) into probabilities by take the exponents of each output and then normalize each number by the sum of those exponents so the entire output vector adds up to one — all probabilities should add up to one.


Softmax function turns logits  into probabilities and the probabilities sum to 1.

In [17]:
import numpy as np

In [18]:
logits = [2.0, 1.0, 0.1]
exps = [np.exp(logit) for logit in logits]
sum_of_exps = sum(exps)
softmax = [j/sum_of_exps for j in exps]
softmax

[0.6590011388859679, 0.2424329707047139, 0.09856589040931818]

In [19]:
sum(softmax)

1.0

In [20]:
def listnet_ce_loss(y_i, z_i): 
    """
    y_i: (n_i, 1) GT  - разметка из данных 
    z_i: (n_i, 1) preds - оценки модели 
    """

    P_y_i = torch.softmax(y_i, dim=0)
    P_z_i = torch.softmax(z_i, dim=0)
    return -torch.sum(P_y_i * torch.log(P_z_i))

def listnet_kl_loss(y_i, z_i): #дивергенция кульбака-лейблера
    """
    y_i: (n_i, 1) GT
    z_i: (n_i, 1) preds
    """
    P_y_i = torch.softmax(y_i, dim=0)
    P_z_i = torch.softmax(z_i, dim=0)
    return -torch.sum(P_y_i * torch.log(P_z_i/P_y_i))


def make_dataset(N_train, N_valid, vector_dim):
    fake_weights = torch.randn(vector_dim, 1)

    X_train = torch.randn(N_train, vector_dim)
    X_valid = torch.randn(N_valid, vector_dim)

    ys_train_score = torch.mm(X_train, fake_weights)
    ys_train_score += torch.randn_like(ys_train_score) #добавляем шум 

    ys_valid_score = torch.mm(X_valid, fake_weights) #создадим таргеты и пошумим на них 
    ys_valid_score += torch.randn_like(ys_valid_score)

#     bins = [-1, 1]  # 3 relevances
    bins = [-1, 0, 1, 2]  # 5 relevances
    ys_train_rel = torch.Tensor(
        np.digitize(ys_train_score.clone().detach().numpy(), bins=bins)
    )
    ys_valid_rel = torch.Tensor(
        np.digitize(ys_valid_score.clone().detach().numpy(), bins=bins)
    )

    return X_train, X_valid, ys_train_rel, ys_valid_rel

In [21]:
N_train = 1000
N_valid = 500

vector_dim = 100
epochs = 2

batch_size = 16 #будет наши шагом градиентного спуска  

X_train, X_valid, ys_train, ys_valid = make_dataset(N_train, N_valid, vector_dim)

net = ListNet(num_input_features=vector_dim)
opt = torch.optim.Adam(net.parameters())


In [22]:
torch.unique(ys_train)

tensor([0., 1., 2., 3., 4.])

In [23]:
for epoch in range(epochs):
    idx = torch.randperm(N_train) #перемешиваем тренировочные данные и получаем новые индексы для тренировки 

    X_train = X_train[idx]
    ys_train = ys_train[idx]

    cur_batch = 0
    for it in range(N_train // batch_size):
        batch_X = X_train[cur_batch: cur_batch + batch_size]
        batch_ys = ys_train[cur_batch: cur_batch + batch_size] #метки на указанный батч 
        cur_batch += batch_size

        opt.zero_grad() #перед обучением зануляем градиенты 
        if len(batch_X) > 0: #проверка, что бантч не пустой 
            batch_pred = net(batch_X) #применяем модель и получаем логиты 
            batch_loss = listnet_kl_loss(batch_ys, batch_pred) #считаем кросс энтропию 
#             batch_loss = listnet_ce_loss(batch_ys, batch_pred)
            batch_loss.backward(retain_graph=True) #считаем градиенты и применяем алогритм Backpropagation
            opt.step() #шаг градиентного спуска 

        if it % 10 == 0: #каждый десятый батч делаем валидацию 
            with torch.no_grad():
                valid_pred = net(X_valid) #скрамливаем всю валидационную группу модели и получаем предсказания 
                valid_swapped_pairs = num_swapped_pairs(ys_valid, valid_pred) #количество неправильно упорядоченных пар 
                ndcg_score = ndcg(ys_valid, valid_pred)
            print(f"epoch: {epoch + 1}.\tNumber of swapped pairs: " 
                  f"{valid_swapped_pairs}/{N_valid * (N_valid - 1) // 2}\t"
                  f"nDCG: {ndcg_score:.4f}")

epoch: 1.	Number of swapped pairs: 34218/124750	nDCG: 0.8830
epoch: 1.	Number of swapped pairs: 31158/124750	nDCG: 0.9030
epoch: 1.	Number of swapped pairs: 28822/124750	nDCG: 0.9139
epoch: 1.	Number of swapped pairs: 26453/124750	nDCG: 0.9240
epoch: 1.	Number of swapped pairs: 24438/124750	nDCG: 0.9333
epoch: 1.	Number of swapped pairs: 22461/124750	nDCG: 0.9406
epoch: 1.	Number of swapped pairs: 20501/124750	nDCG: 0.9499
epoch: 2.	Number of swapped pairs: 20147/124750	nDCG: 0.9513
epoch: 2.	Number of swapped pairs: 18580/124750	nDCG: 0.9567
epoch: 2.	Number of swapped pairs: 16974/124750	nDCG: 0.9627
epoch: 2.	Number of swapped pairs: 15546/124750	nDCG: 0.9674
epoch: 2.	Number of swapped pairs: 14424/124750	nDCG: 0.9712
epoch: 2.	Number of swapped pairs: 13598/124750	nDCG: 0.9737
epoch: 2.	Number of swapped pairs: 12804/124750	nDCG: 0.9760
